# Сравнительный анализ методов имитации отжига и генетических алгоритмов

## Цель работы
Определение предпочтительных методов решения каждой из представленных задач и выделение особенностей работы методов при их решении.
## Набор задач для наиболее показательного сравнения методов.
1. Задача о расстановке ферзей
2. Задача Коммивояжера

## Задача о расстановке ферзей
### Формулировка
Дано шахматное поле размера N × N.
Требуется разместить на нём N ферзей так, чтобы ни одна пара ферзей не угрожала друг другу.
### Решения
1. Полный перебор
2. Перебор перестановок 
3. Динамическое программирование (для очень маленьких досок)
4. Эволюционные алгоритмы
5. Градиентный спуск (отжиг)
### Почему используем отжиг и генетические алгоритмы?
Очевидно, что первые 2 способа решения будут работать за O(N!), что позволит за приемлемое время находить ответ только для очень маленьких досок. Динамическое программирование тут тоже особо не помогает, ибо у нас все равно порядка O(N!) состояний, которые требуется посетить. В то же время эвристические алгоритмы вроде отжига и генетического алгоритма позволяют решить эту задачу для более больших досок. При этом использовать неточные методы нас подталкивает еще и наличие критерия, по которому мы можем определить, устраивает нас ответ или нет (если ферзи друг друга не бьют, то все хорошо, ответ один из лучших).
### Целевая функция
Для обоих алгоритмов она будет одинакова. Но тут встает вопрос: какую же функцию использовать? При использовании бинарной функциии вида: хорошая доска (никто друг друга не бьет), плохая доска, возникает проблема - неинформативность. Алгоритмы банально не смогут сказать, а какое из плохих решений ближе к правильному. Я предлагаю использовать для текущей задачи следующую функцию стоимости: количество пар ферзей, которые друг друга бьют. Данную функцию нужно минимизировать.
### Область решений
Мы могли бы считать областью решений все возможные расстановки ферзей без учета условия о том, что они не должны друг друга бить. Но эта область довольно крупная. Давайте ее сузим. Мы знаем, что задачу можно решить перебором перестановок, и, действительно, ни в одной строке и ни в одном столбце не может стоять более 1 ферзя. Наша область решений - все возможные перестановки чисел от 1 до N.

## Решение задачи о расстановке ферзей методом отжига

### Функция генерации случайного решения
Будет общей для ГА и МО. Представляет собой генерацию случайной перестановки.

In [ ]:
import random


def make_permutation(n: int, rng: random.Random) -> list[int]:
    perm = list(range(n))
    rng.shuffle(perm)
    return perm

### Функции температур
Пердоставлю сразу несколько функций температур для метода отжига, чтобы в дальнейшем сконструировать несколько вариаций алгоритма.

In [ ]:
funcs = []

def temp_func_1(iteration: int) -> float:
    return 1.0 / iteration

def temp_func_2(iteration: int) -> float:
    return 1.0 / (iteration ** 2)

def temp_func_3(iteration: int) -> float:
    return 0.99 ** (iteration - 1)

funcs = [temp_func_1, temp_func_2, temp_func_3]

### Функция мутации
Тут можно использовать очень простую функцию мутации - смена двух случайных индексов.

In [ ]:
def mutate_func(permutation: list[int], rng: random.Random) -> list[int]:
    i, j = rng.sample(range(len(permutation)), 2)
    permutation[i], permutation[j] = permutation[j], permutation[i]
    return permutation

### Целевая функция

In [ ]:
def cost_func(permutation: list[int]) -> float:
    diag1 = [0 for i in range(len(permutation) * 2 - 1)]
    diag2 = [0 for i in range(len(permutation) * 2 - 1)]
    for i in range(len(permutation)):
        diag1[permutation[i] + i] += 1
        diag2[permutation[i] - i + len(permutation) - 1] += 1
    return sum(d * (d-1) / 2 for d in diag1) + sum(d * (d-1) / 2 for d in diag2)

### Класс алгоритма метода отжига
Находится в algorithms/sa.py  
Определим несколько объектов с разными функциями температур

In [ ]:
from sa_ga_comparison.algorithms.sa import SimulatedAnnealing


random_seed = 13

sa = [SimulatedAnnealing(make_permutation, funcs[i], mutate_func, cost_func, random.Random(random_seed)) for i in range(3)]

### Пробуем получить решения
Сгенерируем решения для N = [10, 100, 1000]

In [ ]:
from sa_ga_comparison.utils.benchmarks import run_experiment


for N in [10, 100, 1000]:
    for s in sa:
        res = run_experiment(s, N)
        for j in res[0]:
            print(f"Result for {s}: {j}")
        print("Average time:", res[1])